#Assignment 4

In [2]:
# first, upload alice.txt with Data tab
# then, run it
lines = sc.textFile("/FileStore/tables/alice.txt")

# the number of lines (it should return: 3736)
lines.count()

In [3]:
# we can us it for reduceByKey
from operator import add

# a more refined word splitting than x.split(), with Regular Expressiopns
import re
regex_words = re.compile(r"(\w[\w']*\w|\w)")


words = lines.flatMap(lambda x: regex_words.findall(x)).cache()
words.count()

# Exercises

## Instructions

* Don't remove or modify markdown cells.
* There should be exactly one Python cell per exercise, directly below markdown cell.
* Aesthetics: one operation (e.g. `filter` or `flatMap`) per line.
* No Python loops or Pandas. No Python operations on the results after retriving data from Sparl (`collect`/`take`/`top`). In particular, you cannot write `.collect()[:5]`.
* Don't redefine `lines` or `words`.
* One chain per cell, not saving additional variables.
* Don't ignore case, unless asked explicitly. 
* Don't modify any code above.
* Run all and export data asa  HTML file (as it contains the results) and Jupyter Notebook.

## Example

Count the number of lines with less than 20 characters (including spaces).

In [6]:
lines \
  .filter(lambda x: len(x) < 20) \
  .count()

## Exercise 1

What is the number of **lines** containing phrase `Alice` (i.e. `Alice's` counts, but not - `ALICE`)?

#### Python hint

`"Alice" in "a book called Alice in the Wonderland"`

In [8]:
lines \
  .filter(lambda x: "Alice" in x) \
  .count()

## Exercise 2

Show all lines containing word `mushroom`.

In [10]:
lines \
  .filter(lambda x : 'mushroom' in x) \
  .collect()

## Exercise 3

Show the 10 most popular words starting with a capital letter, as key-value pairs, i.e. something like

```
[(u'I', 543),
 (u'Alice', 398),
 ...]
```

#### Python hints

`"abcde"[3]`

`"abcde".isupper()`

In [12]:
words \
  .filter(lambda x: x[0].isupper())\
  .map(lambda x: (x, 1))\
  .reduceByKey(add)\
  .top(10, lambda (k, v) : v)

## Exercise 4

Show the 10 most common words that have more than 10 characters. Ignore case (i.e. turn everything into lower case). Present the result as key-value pairs.

#### Python hints

`"ABCDE".lower()`

In [14]:
words \
  .map(lambda x : x.lower()) \
  .filter(lambda x: len(x) > 10) \
  .map(lambda x: (x, 1)) \
  .reduceByKey(add) \
  .top(10, lambda (k, v) : v)

## Exercise 5

Take words of length at least 2. 
Count characters that start and end these (i.e. for `Alice` it is `A` and `e`, for `but` it is `b` and `t`), as key-value pairs, sorted alphabetically.

In [16]:
words \
  .filter(lambda x : len(x) >= 2) \
  .flatMap(lambda x : (x[0], x[-1])) \
  .map(lambda x : (x, 1)) \
  .reduceByKey(add) \
  .sortBy(lambda (k, v) : k) \
  .collect()

## Exercise 6

Take words that appear at least 3 times in the text. Show the number of unique words by word length. Present the result as key-value pairs, sorted as follows:

```
[(1, 28),
 (2, 61),
 (3, 147),
 ...]
```

#### Hint

After `.reduceByKey(add)` you are free to use further Spark operations.

In [18]:
words \
  .map(lambda x : (x, 1)) \
  .reduceByKey(add) \
  .filter(lambda (k, v): v >= 3) \
  .map(lambda (k, v) : k) \
  .map(lambda k : (len(k), 1)) \
  .reduceByKey(add) \
  .sortBy(lambda(k, v): k) \
  .collect()